# 📘 Agentic RAG Core Notebook
This notebook implements a LangGraph-based RAG pipeline using modular agents.

In [1]:
# Set base path for project
import os
import sys

base_path = os.path.abspath(os.path.join(os.getcwd()))
sys.path.append(os.path.join(base_path, "src"))

In [2]:
# Install dependencies
!pip install langgraph faiss-cpu sentence-transformers pdfplumber openai



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
%pip install openai==0.28


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
from utils.pdf_utils import extract_text_from_pdf, clean_text, chunk_text
from agents.embedder_agent import EmbedderAgent
from agents.retriever_agent import RetrieverAgent
from agents.qa_agent import QAAgent
from langgraph.graph import StateGraph

## Step 1: Load and Chunk PDF

In [23]:
file_path = f"{base_path}/data/pharmacology_review.pdf"
raw_text = extract_text_from_pdf(file_path)
cleaned = clean_text(raw_text)
chunks = chunk_text(cleaned)


## Step 2: Initialize Agents

In [24]:
import os

print(f"Current base_path: {base_path}")
print(f"Contents of {base_path}:")
try:
    for item in os.listdir(base_path):
        print(item)
except FileNotFoundError:
    print(f"Error: Directory not found at {base_path}")

Current base_path: /Users/samo13loaner05/Desktop/PharmAGen
Contents of /Users/samo13loaner05/Desktop/PharmAGen:
.DS_Store
requirements.txt
02_agentic_rag_core.ipynb
.gitignore
.env
venv
.git
.vscode
data
outputs
notebooks
src


In [25]:
embedder = EmbedderAgent()
retriever = RetrieverAgent(chunks, embedder)
qa_agent = QAAgent()


## Step 3: Build LangGraph

In [26]:
from langgraph.graph import StateGraph
from typing import TypedDict, List

In [27]:
class RAGState(TypedDict):
    query: str
    query_embedding: List[float]
    retrieved_chunks: List[str]
    answer: str

builder = StateGraph(RAGState)

builder.add_node("embedder", embedder.run)
builder.add_node("retriever", retriever.run)
builder.add_node("qa", qa_agent.run)

builder.set_entry_point("embedder")
builder.add_edge("embedder", "retriever")
builder.add_edge("retriever", "qa")

graph = builder.compile()


## Step 4: Query the Graph

In [28]:
query = "What is the half-life of Semaglutide?"
response = graph.invoke({"query": query})
print(response)


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.